# Pytorch基本训练框架

## 基本组件1: 神经网络
1. 所有的Pytorch神经网络都必须继承自一个基类: nn.Module
2. 两个最重要的函数:
    1. 构造函数 __init__ : 定义所有成员变量, 也就是网络结构
    2. forward()函数: 定义网络的前向过程

In [38]:
import torch
import torch.nn as nn

# 定义网络
class TestNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,10)
        )
    def forward(self,x):
        B,C,W,H = x.shape
        x = x.view(B,W*H)
        return self.net(x)

# 初始化网络
model = TestNet().cuda()

## 基本组件2: 优化器
1. 优化器一般不用自己写, 通常继承自: torch.optim.Optimizer
2. 优化器的构造函数中必须定义该优化器对应的可优化参数
3. 例如learning rate, weight decay, momentum 之类的都是不同优化器的可调节参数，这些参数对最终模型的性能影响非常大

In [44]:
from torch.optim import Adam
optimizer = Adam(model.parameters(),lr=1e-3)

## 基本组件3: 损失函数
1. 损失函数可以自己任意定义，一般来说输出是一个标量的损失值

In [13]:
import torch.nn.functional as F
loss_func = F.cross_entropy

## 定义训练函数
有了上述三个组件，我们就可以定义训练过程了，在很多框架中，下面的这个函数被称为trainer

trainer 定义了训练中一个完整的正向、反向传播过程

trainer 不一定有返回值,一般会返回loss的数值进行可视化，在这个过程中，重要的是model这个对象的所有参数获得了更新

In [31]:
# 注意loss_func不一定通过传参形式给到trainer, 可以直接import
# device 是CPU或CUDA, 或者特定编号的GPU
def trainer(batch, model, optimizer, loss_func, device):
    # 将模型参数设为训练模式
    model.train()
    # 从batch中获取输入数据和标签(不一定有标签)
    x, y = batch
    # 将数据存入对应设备中
    x = x.to(device)
    y = y.to(device)
    # 梯度清零
    optimizer.zero_grad()
    # 前向传播
    y_hat = model(x)
    # 计算loss
    loss = loss_func(y_hat,y)
    # 反向传播获取梯度
    loss.backward()
    # 更新参数
    optimizer.step()

    # 计算准确率
    predictions = torch.argmax(y_hat, dim=1)  # 获取模型的预测结果
    correct_predictions = (predictions == y).sum().item()  # 统计正确的预测数量
    return loss.item() / y.shape[0], correct_predictions

## 定义验证函数
1. 验证函数需要将模型设置为预测模式
2. 需要输入validation loader

In [27]:
def validate(val_loader, model, loss_func, device):
    # 将模型参数设为评估模式
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # 禁用梯度计算
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)

            # 前向传播
            y_hat = model(x)
            # 计算loss
            loss = loss_func(y_hat, y)

            total_loss += loss.item() / y.size(0)

            # 计算准确率
            predictions = torch.argmax(y_hat, dim=1)
            correct_predictions += (predictions == y).sum().item()
            total_samples += y.size(0)

    # 计算平均损失和准确率
    average_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_samples

    print(f"Validation Loss: {average_loss:.4f} | Accuracy: {accuracy * 100:.2f}%")

    return average_loss, accuracy

## 输入数据

## 数据集
1. 数据集通常继承自: torch.utils.data.Dataset 基类
2. 对于图像中的常用数据集, 一般会在torchvision库中有定义好的Dataset类
3. 对于不常用的数据集，往往需要手写Dataset类, 手写Dataset类时一定需要的是__getitem__方法

In [28]:
from torch.utils.data import Dataset
from torchvision.datasets import MNIST
import torchvision.transforms as T

# 定义数据集的转换
my_transform = T.Compose([
    T.ToTensor(),  # 将图像转换为张量
    T.Normalize((0.5,), (0.5,))  # 标准化张量，使其范围在[-1, 1]之间
])

# 初始化训练集和测试集
train_dataset = MNIST(root='./data', train=True, transform=my_transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=my_transform, download=True)

# 自定义数据集
class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = [0 for i in range(100)]

    def __getitem__(self, index):
        return self.data[i]

## 数据读取
1. DataLoader可以从数据集中读取不同batch的数据，可以通过定义num_workers来定义数据读取线程的数量, 通过pin_memory来决定数据是否要存放在内存条中

In [42]:
from torch.utils.data import DataLoader

# 定义 DataLoader 来加载数据
batch_size = 512
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

## 开始训练

In [47]:
device = "cuda"
total_ep = 100
for ep in range(total_ep):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        loss, correct_predictions = trainer(batch, model, optimizer, loss_func, device)
        total_loss += loss
        total_correct += correct_predictions
        total_samples += batch[1].shape[0]
    average_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    print(f"Epoch: {ep} Training Loss: {average_loss:.4f} | Accuracy: {accuracy * 100:.2f}%")
        
    if (ep+1) % 5 == 0:
        validate(test_loader, model, loss_func, device)

## 结果输出
1. 可以输出到txt文件
2. 直接使用print
3. 使用tensorboard等工具

# Tips
1. 代码需要遵循“高内聚，低耦合”的设计思路，既每个函数、每个类都只完成最少的任务，这样才方便修改，快速移植
2. 善于组合别人的代码，将别人的模块化用到自己的代码中